# ВОПРОСЫ
1. Нужно ли предсказывать бит ошибки?
2. Предсказываем 30 классов (индексы ошибки)? регрессия или классификация?
3. Как сделать лосс?
4. 


In [1]:
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse
# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()
# import getpass
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [2]:
# !mkdir -p drive
# !google-drive-ocamlfuse drive

In [5]:
import os 
import sys
import tensorflow as tf
import pandas as pd

from tensorflow.python.keras import models
from sklearn import model_selection
from __future__ import absolute_import

# os.chdir("/content/drive/Hamming")
# sys.path.append("Hamming")
# import functions as f
# import model_lib 

In [6]:
tf.test.gpu_device_name()

''

In [7]:
!ls

dataset_files  functions.py	    model_lib.py  visualisation.ipynb
dnn_model      hamming-codes.ipynb  README.md


In [8]:
# !git pull

In [9]:
TRAIN_PATH = './dataset_files/hamming.txt'  # "/content/drive/Hamming/dataset_files/hamming.txt"

COLUMN_NAMES = ['id', 'plainword', 'codeword', 
                'id_error', 'bin_error', 'defective_codeword']

In [10]:
def hamming_distance(first: str, second: str) -> int:
    return len([1 for (x, y) in zip(first, second) if x != y])

In [11]:
def loss_hamming(y, y_pred):  # , w):
    return sum(1 for (a,b) in zip(y, y_pred) if y != y_pred) / y.shape[0] # надо ли делить????

In [12]:
def load_data():
    data = pd.read_csv(TRAIN_PATH, sep=';', names=COLUMN_NAMES)

    # make features
    data['dec_defective_codeword'] = data['defective_codeword'][:].apply(lambda x: int(x, 2))
    for j in range(len(data['defective_codeword'][0])):
        data['bin_' + str(j)] = data['defective_codeword'][:].apply(lambda x: int(x[j]))
    data['dec_plainword'] = data['plainword'][:].apply(lambda x: int(x, 2))
    return data

In [13]:
def split_data(test_size):
    train_data, test_data, train_labels, test_labels = \
    model_selection.train_test_split(data.loc[:, 'bin_0':'bin_30'], 
                                        data['id_error'], 
                                        test_size = test_size)
    return train_data, test_data, train_labels, test_labels

In [14]:
data = load_data() #pd.read_csv(functions.TRAIN_PATH, sep=';', names = functions.COLUMN_NAMES)

In [15]:
data.head()

,id,plainword,codeword,id_error,bin_error,defective_codeword,dec_defective_codeword,bin_0,bin_1,bin_2,...,bin_22,bin_23,bin_24,bin_25,bin_26,bin_27,bin_28,bin_29,bin_30,dec_plainword
0,0,10110000001111101100000100,0001001100000011111011000001000,8,0000000010000000000000000000000,0001001110000011111011000001000,163706376,0,0,0,...,0,0,0,0,0,1,0,0,0,46201604
1,1,00101011010000101000111100,1001101010110100001010001111000,17,0000000000000000010000000000000,1001101010110100011010001111000,1297757304,1,0,0,...,0,0,1,1,1,1,0,0,0,11340348
2,2,10010100110111111111111100,1101100101001101111111111111000,18,0000000000000000001000000000000,1101100101001101110111111111000,1822879736,1,1,0,...,1,1,1,1,1,1,0,0,0,39026684
3,3,00011000001101000100010111,1011100110000011010001000101110,13,0000000000000100000000000000000,1011100110000111010001000101110,1556324910,1,0,1,...,0,0,0,1,0,1,1,1,0,6344983
4,4,10011101101000001100111011,0011000111011010000011001110110,19,0000000000000000000100000000000,0011000111011010000111001110110,418188918,0,0,1,...,0,0,1,1,1,0,1,1,0,41321275


In [16]:
def make_features():
  data['dec_defective_codeword'] = data['defective_codeword'][:].apply(lambda x: int(x, 2))

  for j in range(len(data['defective_codeword'][0])):
    data['bin_' + str(j)] = data['defective_codeword'][:].apply(lambda x: int(x[j]))
  
  data['dec_plainword'] = data['plainword'][:].apply(lambda x: int(x, 2))

In [17]:
make_features()

In [18]:
data.head()

,id,plainword,codeword,id_error,bin_error,defective_codeword,dec_defective_codeword,bin_0,bin_1,bin_2,...,bin_22,bin_23,bin_24,bin_25,bin_26,bin_27,bin_28,bin_29,bin_30,dec_plainword
0,0,10110000001111101100000100,0001001100000011111011000001000,8,0000000010000000000000000000000,0001001110000011111011000001000,163706376,0,0,0,...,0,0,0,0,0,1,0,0,0,46201604
1,1,00101011010000101000111100,1001101010110100001010001111000,17,0000000000000000010000000000000,1001101010110100011010001111000,1297757304,1,0,0,...,0,0,1,1,1,1,0,0,0,11340348
2,2,10010100110111111111111100,1101100101001101111111111111000,18,0000000000000000001000000000000,1101100101001101110111111111000,1822879736,1,1,0,...,1,1,1,1,1,1,0,0,0,39026684
3,3,00011000001101000100010111,1011100110000011010001000101110,13,0000000000000100000000000000000,1011100110000111010001000101110,1556324910,1,0,1,...,0,0,0,1,0,1,1,1,0,6344983
4,4,10011101101000001100111011,0011000111011010000011001110110,19,0000000000000000000100000000000,0011000111011010000111001110110,418188918,0,0,1,...,0,0,1,1,1,0,1,1,0,41321275


In [19]:
train_data, test_data, train_labels, test_labels = split_data(test_size=0.3)

In [20]:
# Create feature columns for all features.
feature_columns = []
for key in train_data.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key))

In [40]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[100, 100, 100],
    n_classes=train_data.shape[1]
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp20u51pdk', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbb703c6278>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [49]:
train_input_fn = tf.estimator.inputs.pandas_input_fn(x=train_data,
                                                    y=train_labels,
                                                    num_epochs=3,
                                                    batch_size=100,
                                                    shuffle=True
                                                    )

In [55]:
test_input_fn = tf.estimator.inputs.pandas_input_fn(x=test_data,
                                                    y=test_labels,
                                                    num_epochs=3,
                                                    batch_size=100,
                                                    shuffle=True
                                                    )

In [58]:
classifier.train(input_fn=train_input_fn, steps=100000000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp20u51pdk/model.ckpt-1377
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1378 into /tmp/tmp20u51pdk/model.ckpt.
INFO:tensorflow:loss = 339.55203, step = 1378
INFO:tensorflow:global_step/sec: 152.142
INFO:tensorflow:loss = 338.5431, step = 1478 (0.658 sec)
INFO:tensorflow:global_step/sec: 176.371
INFO:tensorflow:loss = 337.81427, step = 1578 (0.567 sec)
INFO:tensorflow:global_step/sec: 183.827
INFO:tensorflow:loss = 339.4119, step = 1678 (0.544 sec)
INFO:tensorflow:global_step/sec: 206.902
INFO:tensorflow:loss = 338.39435, step = 1778 (0.483 sec)
INFO:tensorflow:global_step/sec: 178.518
INFO:tensorflow:loss = 339.16812, step = 1878 (0.560 sec)
INFO:tensorflow:global_step/sec: 175.883
INFO:tensorflow:loss = 340.652

In [59]:
accuracy_score = classifier.evaluate(input_fn=test_input_fn)['accuracy']
print('\nTest Accuracy: {0:f}%\n'.format(accuracy_score*100))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-03-04-11:03:18
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp20u51pdk/model.ckpt-2754
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-03-04-11:03:20
INFO:tensorflow:Saving dict for global step 2754: accuracy = 0.031839684, average_loss = 3.4141254, global_step = 2754, loss = 341.31418

Test Accuracy: 3.183968%

